In [24]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Воспроизводимость
RANDOM_STATE = 42

# Библиотеки sklearn для данных, моделей, разбиения и метрик
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    RocCurveDisplay,
    PrecisionRecallDisplay,
    brier_score_loss,
)

# Для калибровки
from sklearn.calibration import CalibratedClassifierCV

In [31]:
# загрузка датасета
url = 'S05-hw-dataset.csv' 
data = pd.read_csv(url, sep = ',')
data

,client_id,age,income,years_employed,credit_score,debt_to_income,num_credit_cards,num_late_payments,has_mortgage,has_car_loan,savings_balance,checking_balance,region_risk_score,phone_calls_to_support_last_3m,active_loans,customer_tenure_years,default
0,1,25,94074,22,839,0.547339,1,7,0,0,26057,5229,0.080052,19,1,8,0
1,2,58,51884,26,565,0.290882,1,1,0,1,16221,11595,0.428311,15,0,7,0
2,3,53,48656,39,561,0.522340,1,13,0,0,55448,-2947,0.770883,15,4,5,0
3,4,42,81492,30,582,0.709123,2,10,1,1,35188,17727,0.357619,0,2,7,1
4,5,42,94713,8,642,0.793392,3,3,0,0,0,-404,0.414260,17,1,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,51,131729,6,687,0.283093,3,1,1,0,13136,14209,0.251680,19,3,2,0
2996,2997,35,54454,36,634,0.396180,5,5,0,0,0,10858,0.506146,2,2,12,0
2997,2998,38,63305,16,573,0.577888,2,13,0,1,24870,6689,0.366375,16,2,10,1
2998,2999,42,55606,33,689,0.659278,6,9,1,0,32677,3252,0.295284,16,2,8,1


In [6]:
data.head(5) # вывод первых 5 строк

,client_id,age,income,years_employed,credit_score,debt_to_income,num_credit_cards,num_late_payments,has_mortgage,has_car_loan,savings_balance,checking_balance,region_risk_score,phone_calls_to_support_last_3m,active_loans,customer_tenure_years,default
0,1,25,94074,22,839,0.547339,1,7,0,0,26057,5229,0.080052,19,1,8,0
1,2,58,51884,26,565,0.290882,1,1,0,1,16221,11595,0.428311,15,0,7,0
2,3,53,48656,39,561,0.522340,1,13,0,0,55448,-2947,0.770883,15,4,5,0
3,4,42,81492,30,582,0.709123,2,10,1,1,35188,17727,0.357619,0,2,7,1
4,5,42,94713,8,642,0.793392,3,3,0,0,0,-404,0.414260,17,1,10,1


In [ ]:
data.info() # информация о столбцах и типах

In [ ]:
data.describe() # описательная статистика

In [18]:
data["default"].value_counts(normalize=True) # распределение value_counts

default
0    0.589667
1    0.410333
Name: proportion, dtype: float64

In [ ]:
# подготовка признаков таргета

X = data.drop(['default', "client_id"], axis=1)
Y = data['default']
print(X.shape)
print(Y.shape)


(3000, 15)
(3000,)


In [ ]:
# разбиение на обучающую и тестовую выборку

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42, stratify = Y)
print("Обучающие данные:",X_train.shape, Y_train.shape)
print("Тестовые данные:",X_test.shape, Y_test.shape)

Обучающие данные: (2400, 15) (2400,)
Тестовые данные: (600, 15) (600,)


In [11]:
# реализация "тупой" модели

dummy = DummyClassifier(strategy = "most_frequent", random_state = 42)
dummy.fit(X_train, Y_train)

Y_pred = dummy.predict(X_test)
Y_proba = dummy.predict_proba(X_test)[:, 1]

dummy_acc = accuracy_score(Y_test, Y_pred)
dummy_roc = roc_auc_score(Y_test, Y_proba)

print(f"dummy: accuracy = {dummy_acc:.4f}, ROC-AUC = {dummy_roc:.4f}")

dummy: accuracy = 0.5900, ROC-AUC = 0.5000


In [16]:
# создание пайплайна

pipe = Pipeline(
    steps = [("scaler", StandardScaler()), 
    ("logreg", LogisticRegression(max_iter=1000, random_state=42))]
)
pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('logreg', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not w

In [28]:
# подбор двух парамсетров при помощи GridSearchCV: параметра регуляризации (С) и class_weight(дисбаланс классов). 

param_grid = {"logreg__C": [0.01, 0.1, 1.0, 10.0], 
              "logreg__class_weight": [None, "balanced"]}

grid = GridSearchCV (estimator=pipe, 
                     param_grid=param_grid, 
                     scoring="roc_auc",
                     cv=5,
                     n_jobs=-1, 
                     refit=True)

grid.fit(X_train, Y_train)

print("Лучшие параметры:", grid.best_params_)
print("Лучише CV и ROC-AUC:", grid.best_score_)

best_model = grid.best_estimator_

Y_pred = best_model.predict(X_test)
Y_proba = best_model.predict_proba(X_test)[:, 1]

acc = accuracy_score(Y_test, Y_pred)
roc = roc_auc_score(Y_test, Y_proba)
prec = precision_score(Y_test, Y_pred)
rec = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print(f"Метрики на тестовой выборке| accuracy={acc:.4f} | roc_auc={roc:.4f} | precision={prec:.4f} | recall={rec:.4f} | f1={f1:.4f}")

Лучшие параметры: {'logreg__C': 10.0, 'logreg__class_weight': None}
Лучише CV и ROC-AUC: 0.8606590016322577
Метрики на тестовой выборке| accuracy=0.8000 | roc_auc=0.8756 | precision=0.7944 | recall=0.6911 | f1=0.7391


In [ ]:
# создание ROC графика

figures_dir = Path("figures")
figures_dir.mkdir(exist_ok=True)
plt.figure()
RocCurveDisplay.from_predictions(Y_test, Y_proba)
plt.title("ROC кривая лучшей модели логистической регрессии")
plt.savefig(figures_dir / "roc_curve_logreg_grid.png", dpi=150, bbox_inches="tight")
plt.close()

<Figure size 640x480 with 0 Axes>

In [ ]:
# сравнение параметров бейзлайна и логистической регрессии

comparison_df = pd.DataFrame(
    {
        "Модель": ["DummyClassifier", "Логистическая регрессия"],
        "Accurecy": [dummy_acc, acc],
        "ROC-AUC": [dummy_roc, roc]
    }
)
comparison_df

,Модель,Accurecy,ROC-AUC
0,DummyClassifier,0.59,0.500000
1,Логистическая регрессия,0.80,0.875637


Бейзлайн модель не использует признаки и предсказывает наиболее частый класс. Ее значение ROC-AUC близко к 0.5, что соответсвует случайному угадыванию, в то время как логистическая регрессия показала себя намного лучше. Значение accurecy выросло с 0.59 до 0.8, а ROC-AUC с 0.50 до 0.876. Подбор параметра регуляризации С при помощи GridSerachCV позволил улучшить качество модели. Логистическая регрессия более разумный выбор, так как обеспечила более высокое качество.